In [1]:
import ambulance_game as abg
import numpy as np
import random
import ciw
import collections
import scipy.optimize


In [348]:
def extract_times_from_records(simulation_records, warm_up_time):
    waiting = [
        r.waiting_time
        for r in simulation_records
        if r.arrival_date > warm_up_time and r.node == 2
    ]
    serving = [
        r.service_time
        for r in simulation_records
        if r.arrival_date > warm_up_time and r.node == 2
    ]
    blocking = [
        r.time_blocked
        for r in simulation_records
        if r.arrival_date > warm_up_time and r.node == 1
    ]
    return waiting, serving, blocking


def get_list_of_results(results):
    all_waits = [w.waiting_times for w in results]
    all_services = [s.service_times for s in results]
    all_blocks = [b.blocking_times for b in results]
    return all_waits, all_services, all_blocks


def get_multiple_runs_results(
    lambda_2,
    lambda_1,
    mu,
    num_of_servers,
    threshold,
    seed_num=None,
    warm_up_time=100,
    num_of_trials=10,
    runtime=1440,
    output_type="tuple",
    system_capacity=float("inf"),
    buffer_capacity=float("inf"),
    patient_type="both",
):

    if seed_num == None:
        seed_num = random.random()
    records = collections.namedtuple(
        "records", "waiting_times service_times blocking_times"
    )
    results = []
    for trial in range(num_of_trials):
        simulation = abg.simulation.simulate_model(
            lambda_2,
            lambda_1,
            mu,
            num_of_servers,
            threshold,
            seed_num + trial,
            runtime,
            system_capacity,
            buffer_capacity,
        )

        if patient_type == "both":
            sim_results = simulation.get_all_records()
            ext = extract_times_from_records(sim_results, warm_up_time)
            results.append(records(ext[0], ext[1], ext[2]))
        elif patient_type == "ambulance":
            individuals = simulation.get_all_individuals()
            ext = extract_times_from_individuals(
                individuals, warm_up_time, first_node_to_visit=1, total_node_visits=2
            )
            #             return ext
            results.append(records(ext[0], ext[1], ext[2]))
        elif patient_type == "others":
            individuals = simulation.get_all_individuals()
            ext = extract_times_from_individuals(
                individuals, warm_up_time, first_node_to_visit=2, total_node_visits=1
            )
            results.append(records(ext[0], ext[1], ext[2]))

    if output_type == "list":
        all_waits, all_services, all_blocks = get_list_of_results(results)
        return [all_waits, all_services, all_blocks]

    return results


In [374]:
def extract_times_from_individuals(
    individuals, warm_up_time, first_node_to_visit, total_node_visits
):
    """
    For all individuals' records
        if not still in the system and after warm_up_time
            
        if finished only dummy service
    """
    waiting = []
    serving = []
    blocking = []

    for ind in individuals:
        if (
            ind.data_records[0].node == first_node_to_visit
            and len(ind.data_records) == total_node_visits
            and ind.data_records[total_node_visits - 1].arrival_date > warm_up_time
        ):
            waiting.append(ind.data_records[total_node_visits - 1].waiting_time)
            serving.append(ind.data_records[total_node_visits - 1].service_time)
        if (
            first_node_to_visit == ind.data_records[0].node == 1
            and ind.data_records[0].arrival_date > warm_up_time
        ):
            blocking.append(ind.data_records[0].time_blocked)

    #     blocking = [
    #         r.time_blocked
    #         for r in simulation_records
    #         if r.arrival_date > warm_up_time and r.node == 1
    #     ]

    return waiting, serving, blocking


#         if ind.data_records[0].node == 1 and len(ind.data_records) == 2:
#             ambulance_patients_times.append(ind.data_records[1].waiting_time)
#             if ind.data_records[0].waiting_time != 0:
#                 raise SystemError('Waiting occurs on the dummy node whilw only blocking should occur')
#         elif ind.data_records[0].node == 2 and len(ind.data_records) == 1:
#             other_patients_times.append(ind.data_records[0].waiting_time)
#         else:
#             patients_still_in_system.append(ind)


In [536]:
lambda_2 = 0.2
lambda_1 = 0.4
mu = 0.2
num_of_servers = 5
threshold = 4
seed_num = 5
warm_up_time = 10
num_of_trials = 20
runtime = 100

output_type = "tuple"
system_capacity = float("inf")
buffer_capacity = float("inf")

patient_type = "both"


In [537]:
demo1 = get_multiple_runs_results(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    seed_num=seed_num,
    warm_up_time=warm_up_time,
    num_of_trials=num_of_trials,
    runtime=runtime,
    output_type=output_type,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    patient_type=patient_type,
)


In [538]:
np.mean([np.mean(w.waiting_times) for w in demo1])


0.12299700568462768

In [490]:
demo2 = get_multiple_runs_results(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    seed_num=seed_num,
    warm_up_time=warm_up_time,
    num_of_trials=num_of_trials,
    runtime=runtime,
    output_type=output_type,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    patient_type="ambulance",
)


In [539]:
np.mean([np.mean(w.waiting_times) for w in demo2])


0.0

In [540]:
demo3 = get_multiple_runs_results(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    seed_num=seed_num,
    warm_up_time=warm_up_time,
    num_of_trials=num_of_trials,
    runtime=runtime,
    output_type=output_type,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
    patient_type="others",
)


In [544]:
[np.mean(w.blocking_times) for w in demo3]


[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [535]:
np.all([0, 0, 0, 0, 2]) == 0


True